In [1]:
from data import get_formatted_datasets
import os

/data/workspace/miniconda3/envs/moa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path ="/data/workspace/projects/moe/datasets/eval_code"
data_name = "humaneval"
data_path_1 = os.path.join(data_path, data_name)
data_name = data_name.lower()
# Load and format datasets
formatted_datasets = get_formatted_datasets(data_path=data_path_1, prompt_only=True)

Datasets: DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test'],
        num_rows: 164
    })
})
Example: {'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'entry_point': 'has_close_elements', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    a

Map: 100%|██████████| 164/164 [00:00<00:00, 17324.41 examples/s]

Formatted datasets: DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test', 'instruction', 'data_name', 'text'],
        num_rows: 164
    })
})
Formatted example: {'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'entry_point': 'has_close_elements', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n  

In [5]:
predict_file = '/data/workspace/projects/moe/AdaMoLE/outputs/llama-3-1-8b-instruct-lora-codealpaca20k/predictions/humaneval_responses.jsonl'
mbpp_file = '/data/workspace/projects/moe/AdaMoLE/outputs/llama-3-1-8b-instruct-lora-codealpaca20k/predictions/mbpp_responses.jsonl'

In [6]:
from code_utils.env import get_feedback, is_correct
import json

In [7]:
with open(predict_file, 'r') as f:
# with open(mbpp_file, 'r') as f:
    lines = f.readlines()
predicts = [json.loads(line) for line in lines]

In [8]:
predicts[0]

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'entry_point': 'has_close_elements',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9,

#  eval code from zhenghaoyu

In [9]:
# humaneval
correct_count = 0
for predict in predicts:
    code = predict['response']
    test = predict['test']
    correct = is_correct(code, test)
    if correct:
        correct_count += 1

print(correct_count/len(predicts))

0.6402439024390244


In [4]:
# mbpp
correct_count = 0
for predict in predicts:
    code = predict['response']
    test = predict['test_list']
    is_passing, feedback = get_feedback(code, test)
    if is_passing:
        correct_count += 1
    else:
        pass
        # print(feedback)

print(correct_count/len(predicts))


0.016


In [ ]:
print(is_correct())

# Human eval 代码评测

In [14]:
from human_eval.data import write_jsonl

In [16]:
predicts = [json.loads(line) for line in lines]
samples = []
for x in predicts:
    sample = {}
    sample['task_id'] = x['task_id']
    sample['completion'] = x['response']
    samples.append(sample)
write_jsonl('sample.jsonl', samples)

In [10]:
! evaluate_functional_correctness sample.jsonl

Reading samples...
164it [00:00, 46653.95it/s]
Running test suites...
100%|████████████████████████████████████████| 164/164 [00:00<00:00, 164.04it/s]
Writing results to sample.jsonl_results.jsonl...
100%|█████████████████████████████████████| 164/164 [00:00<00:00, 107127.53it/s]
{'pass@1': np.float64(0.29878048780487804)}


# get mbpp dataset

In [23]:
import datasets
import json

data_source = "Muennighoff/mbpp"

dataset = datasets.load_dataset(data_source)

test = dataset["test"]

In [36]:
filtered = test.filter(lambda x: 11 <= x["task_id"] <= 510)
filtered


Dataset({
    features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
    num_rows: 500
})

In [38]:
filtered.to_json("/data/workspace/projects/moe/datasets/eval_code/mbpp/test.jsonl", lines=True)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 115.03ba/s]


287032

In [34]:
test

Dataset({
    features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
    num_rows: 974
})